In [1]:
!pip install wandb -qqq
import wandb
import pprint

In [5]:
import pandas as pd
import numpy
import csv

def train_com(config=None):
    
    column_name = ['기업명', '기업코드', '날짜','시가', '고가', '저가', '종가', '거래량'] # 데이터의 열 지정
    result = {}
    
     # Start a new run, tracking hyperparameters in config
    with wandb.init(config=config):
        config=wandb.config

        dataframe = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\단순감성분석\컴투스_train.csv',encoding='cp949') # 코로나 대폭락을 제외한 데이터
        data = dataframe.values.tolist()

        new_dataframe = pd.DataFrame(data, columns=column_name) # 데이터에 열 지정
        new_dataframe = new_dataframe.dropna(axis=0)
        new_dataframe = new_dataframe.reset_index()


        next_day = 0


        buyline = []
        sellLine = []

        while next_day < len(new_dataframe)-config.n_period:
            HH = 0
            LC = 0
            HC = 0
            LL = 0

            HH_array = []
            LC_array = []
            HC_array = []
            LL_array = []

            before_day = next_day + 1

            for n in range(config.n_period):
                HH_array.append(new_dataframe['고가'][before_day+n])
                LC_array.append(new_dataframe['종가'][before_day+n])
                HC_array.append(new_dataframe['종가'][before_day+n])
                LL_array.append(new_dataframe['저가'][before_day+n])


            HH = max(HH_array)
            LC = min(LC_array)
            HC = max(HC_array)
            LL = min(LL_array)

            trigger = max(HH - LC , HC - LL) # 0 = 다음날 1 = 이전날

            buyline_temp = new_dataframe['시가'][next_day] + config.K_long * trigger
            sellLine_temp = new_dataframe['시가'][next_day] - config.k_short * trigger

            buyline.append(buyline_temp)
            sellLine.append(sellLine_temp)


            next_day = next_day+1



        for n in range(config.n_period):
            buyline.append(0)
            sellLine.append(0)



        new_dataframe.insert(0,"longLine",buyline,True) # 다음날 대비 등락률율을 해당날에 넣었음 . 즉 , 당일 기사와 당일 등락률을 비교!
        new_dataframe.insert(0,"shortLine",sellLine,True) # 다음날 대비 등락률율을 해당날에 넣었음 . 즉 , 당일 기사와 당일 등락률을 비교!

        idx_positive = new_dataframe[new_dataframe['longLine']==0].index
        new_dataframe = new_dataframe.drop(idx_positive) # 떨어진 날의 데이터를 제거)


        # test

        # 1천만원
        account = 10000000
        buy_num = 0
        sell_num = 0
        sell_price = 0


        buy = False

        buy_price_all = []
        sell_price_all = []


        # i = 0
        # k = 0

        for i in range(len(new_dataframe)-1,0,-1):

            if(new_dataframe['shortLine'][i] == new_dataframe['longLine'][i]):
                sell_price_all.append(new_dataframe['시가'][i])
                buy_price_all.append(new_dataframe['시가'][i])
                continue

            if(int(new_dataframe['shortLine'][i]) == 0):
                print("0000")
                continue

           # 위 line을 넘어가면 buy
            if(new_dataframe['고가'][i] >= new_dataframe['longLine'][i]):
        #         if(short):
        #             account = new_dataframe['shortLine'][i] * short_num + left
                if(buy==False):
                    buy_num = account//int(new_dataframe['longLine'][i])
                    left = account - buy_num * int(new_dataframe['longLine'][i])
                    print("======================")
                    print(new_dataframe["날짜"][i])
                    print("구매가 : ",new_dataframe['longLine'][i])
                    buy_price_all.append(new_dataframe['longLine'][i])
                    sell_price_all.append(new_dataframe['시가'][i])
                    print("구매갯수 : ", buy_num)
                    print("현재가격 :", buy_num * int(new_dataframe['longLine'][i]))
                    print("나머지 : ", left)
                    print("======================")
                    buy = True
                else :
                    sell_price_all.append(new_dataframe['시가'][i])
                    buy_price_all.append(new_dataframe['시가'][i])

                      # 아래 line을 넘어가면 sell
            elif(new_dataframe['저가'][i] <= new_dataframe['shortLine'][i]):
                if(buy):
                    buy = False
                    sell_price = buy_num * int(new_dataframe['shortLine'][i])
                    account = sell_price * 0.975 + left
                    print("======================")
                    print(new_dataframe["날짜"][i])
                    print("판매가 : ",new_dataframe['shortLine'][i])
                    sell_price_all.append(new_dataframe['shortLine'][i])
                    buy_price_all.append(new_dataframe['시가'][i])
                    print("판매갯수 : ", buy_num)
                    print("현재잔고 : ", account)
                    print("======================")
                else :
                    sell_price_all.append(new_dataframe['시가'][i])
                    buy_price_all.append(new_dataframe['시가'][i])
                

            else:
                sell_price_all.append(new_dataframe['시가'][i])
                buy_price_all.append(new_dataframe['시가'][i])


        #     i+=1

        #         sell = True

        #         short_num = account//int(new_dataframe['shortLine'][i])
        #         short_price = int(new_dataframe['shortLine'][i])
        #         left = account - short_num * int(new_dataframe['shortLine'][i])
        #         print("short",short_num)

        sell_price_all.reverse()
        buy_price_all.reverse()
        print(round(account,2))
        wandb.log({"account": account})
#         result[stock] = round(account,2)


In [6]:
sweep_config = {
    'method': 'bayes'
}

metric={
      'goal': 'maximize',
      'name': 'account'
}

sweep_config['metric'] = metric

parameters_dict = {
}

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'n_period': {
        'max': 5, # 1 ~ 갯수의 평균x2 까지
        'min': 3
    },
      'K_long': {
        'distribution': 'uniform',
        'max': 2.0, # 평균 비율 ~ X2 까지
        'min': 1.0
      },
      'k_short': {
        'distribution': 'uniform',
        'max': 2.0 , # 1 ~ 평균 까지
        'min': 1.0
      }
})


pprint.pprint(sweep_config)

sweep_id = wandb.sweep(sweep_config, project=f"dual_컴투스_일단위_논문")

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'account'},
 'parameters': {'K_long': {'distribution': 'uniform', 'max': 2.0, 'min': 1.0},
                'k_short': {'distribution': 'uniform', 'max': 2.0, 'min': 1.0},
                'n_period': {'max': 5, 'min': 3}}}
Create sweep with ID: zgqbbw6l
Sweep URL: https://wandb.ai/yeongu/dual_%EC%BB%B4%ED%88%AC%EC%8A%A4_%EC%9D%BC%EB%8B%A8%EC%9C%84_%EB%85%BC%EB%AC%B8/sweeps/zgqbbw6l


In [7]:
wandb.agent(sweep_id, train_com, count=100)

wandb: Agent Starting Run: udhp15ek with config:
wandb: 	K_long: 1.8052723695735833
wandb: 	k_short: 1.4823290625552532
wandb: 	n_period: 3
wandb: wandb version 0.10.33 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


20101013
구매가 :  11466.778188721482
구매갯수 :  872
현재가격 : 9998352
나머지 :  1648
20110215
판매가 :  10503.845896225468
판매갯수 :  872
현재잔고 :  8931298.6
20110309
구매가 :  10381.676963568196
구매갯수 :  860.0
현재가격 : 8927660.0
나머지 :  3638.5999999996275
20110519
판매가 :  9410.995661545316
판매갯수 :  860.0
현재잔고 :  7893923.6
20110727
구매가 :  12462.108947829433
구매갯수 :  633.0
현재가격 : 7888446.0
나머지 :  5477.5999999996275
20111020
판매가 :  23461.645743983874
판매갯수 :  633.0
현재잔고 :  14485020.274999999
20120316
구매가 :  18643.87941387477
구매갯수 :  776.0
현재가격 : 14466968.0
나머지 :  18052.27499999851
20120323
판매가 :  17957.35922870823
판매갯수 :  776.0
현재잔고 :  13604318.474999998
20120806
구매가 :  41679.35001151674
구매갯수 :  326.0
현재가격 : 13587354.0
나머지 :  16964.474999997765
20120904
판매가 :  52501.23676259132
판매갯수 :  326.0
현재잔고 :  16704407.324999997
20130305
구매가 :  46898.91030457935
구매갯수 :  356.0
현재가격 : 16695688.0
나머지 :  8719.324999997392
20130425
판매가 :  47624.3728783097
판매갯수 :  356.0
현재잔고 :  16539009.724999998
20140213
구매가 :  25597.13983853402
구매갯

account,59450709.8
_runtime,8
_timestamp,1625321858
_step,0


account,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: 388t7pbt with config:
wandb: 	K_long: 1.3990608438188399
wandb: 	k_short: 1.4188462426219024
wandb: 	n_period: 4
wandb: wandb version 0.10.33 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

account,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: dygkap9n with config:
wandb: 	K_long: 1.0529949367241462
wandb: 	k_short: 1.6358532006472863
wandb: 	n_period: 5
wandb: wandb version 0.10.33 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


20100318
구매가 :  13682.631073426557
구매갯수 :  730
현재가격 : 9987860
나머지 :  12140
20110215
판매가 :  10290.090244054996
판매갯수 :  730
현재잔고 :  7336047.5
20110309
구매가 :  10160.197974689658
구매갯수 :  722.0
현재가격 : 7335520.0
나머지 :  527.5
20111214
판매가 :  18792.385224365647
판매갯수 :  722.0
현재잔고 :  13229155.9
20120116
구매가 :  22395.748240526736
구매갯수 :  590.0
현재가격 : 13213050.0
나머지 :  16105.900000000373
20140825
판매가 :  116009.49387114847
판매갯수 :  590.0
현재잔고 :  66750283.15
20140925
구매가 :  137985.20862289766
구매갯수 :  483.0
현재가격 : 66646755.0
나머지 :  103528.14999999851
20161109
판매가 :  86929.97803656939
판매갯수 :  483.0
현재잔고 :  41040567.474999994
20170208
구매가 :  99044.46708870695
구매갯수 :  414.0
현재가격 : 41004216.0
나머지 :  36351.47499999404
20171107
판매가 :  129166.39271650465
판매갯수 :  414.0
현재잔고 :  52174207.37499999
20171211
구매가 :  130370.27417729315
구매갯수 :  400.0
현재가격 : 52148000.0
나머지 :  26207.37499999255
20181211
판매가 :  130039.78355566318
판매갯수 :  400.0
현재잔고 :  50741417.37499999
20190208
구매가 :  119891.4721519828
구매갯수 :  423.0
현재

account,48188041.325
_runtime,8
_timestamp,1625324719
_step,0


account,▁
_runtime,▁
_timestamp,▁
_step,▁


In [27]:
#test 

column_name = ['기업명', '기업코드', '날짜' ,'시간','시가', '고가', '저가', '종가', '거래량'] # 데이터의 열 지정

    
dataframe = pd.read_csv(rf'C:\Users\Desktop\Desktop\주가예측\주식데이터\유한양행_test.csv',encoding='cp949') # 코로나 대폭락을 제외한 데이터
data = dataframe.values.tolist()

new_dataframe = pd.DataFrame(data, columns=column_name) # 데이터에 열 지정
new_dataframe = new_dataframe.dropna(axis=0)
new_dataframe = new_dataframe.reset_index()


next_day = 0

n_period = 13
K_long = 1.881
k_short = 1.514

buyline = []
sellLine = []

while next_day < len(new_dataframe)-n_period:
    HH = 0
    LC = 0
    HC = 0
    LL = 0

    HH_array = []
    LC_array = []
    HC_array = []
    LL_array = []

    before_day = next_day + 1

    for n in range(n_period):
        HH_array.append(new_dataframe['고가'][before_day+n])
        LC_array.append(new_dataframe['종가'][before_day+n])
        HC_array.append(new_dataframe['종가'][before_day+n])
        LL_array.append(new_dataframe['저가'][before_day+n])


    HH = max(HH_array)
    LC = min(LC_array)
    HC = max(HC_array)
    LL = min(LL_array)

    trigger = max(HH - LC , HC - LL) # 0 = 다음날 1 = 이전날

    buyline_temp = new_dataframe['시가'][next_day] + K_long * trigger
    sellLine_temp = new_dataframe['시가'][next_day] - k_short * trigger

    buyline.append(buyline_temp)
    sellLine.append(sellLine_temp)


    next_day = next_day+1



for n in range(n_period):
    buyline.append(0)
    sellLine.append(0)



new_dataframe.insert(0,"longLine",buyline,True) # 다음날 대비 등락률율을 해당날에 넣었음 . 즉 , 당일 기사와 당일 등락률을 비교!
new_dataframe.insert(0,"shortLine",sellLine,True) # 다음날 대비 등락률율을 해당날에 넣었음 . 즉 , 당일 기사와 당일 등락률을 비교!

idx_positive = new_dataframe[new_dataframe['longLine']==0].index
new_dataframe = new_dataframe.drop(idx_positive) # 떨어진 날의 데이터를 제거)


# test

# 1천만원
account = 10000000
buy_num = 0
sell_num = 0
sell_price = 0


buy = False

buy_price_all = []
sell_price_all = []


# i = 0
# k = 0

for i in range(len(new_dataframe)-1,0,-1):

    if(new_dataframe['shortLine'][i] == new_dataframe['longLine'][i]):
        sell_price_all.append(new_dataframe['시가'][i])
        buy_price_all.append(new_dataframe['시가'][i])
        continue

    if(int(new_dataframe['shortLine'][i]) == 0):
        print("0000")
        continue
        
        
       # 위 line을 넘어가면 buy
    if(new_dataframe['고가'][i] >= new_dataframe['longLine'][i]):
#         if(short):
#             account = new_dataframe['shortLine'][i] * short_num + left

        if(buy==False):
            buy_num = account//(int(new_dataframe['longLine'][i]) * 1.00015)
            left = account - buy_num * int(new_dataframe['longLine'][i]) * 1.00015
            print("======================")
            print(new_dataframe["날짜"][i])
            print(new_dataframe["시간"][i])
            print("구매가 : ",new_dataframe['longLine'][i] * 1.00015)
            buy_price_all.append(new_dataframe['longLine'][i])
            sell_price_all.append(new_dataframe['시가'][i])
            print("구매갯수 : ", buy_num)
            print("현재가격 :", buy_num * int(new_dataframe['longLine'][i])* 1.00015)
            print("나머지 : ", left)
            print("======================")
            buy = True
        
        else :
            sell_price_all.append(new_dataframe['시가'][i])
            buy_price_all.append(new_dataframe['시가'][i])    

              # 아래 line을 넘어가면 sell
    elif(new_dataframe['저가'][i] <= new_dataframe['shortLine'][i]):
        if(buy):
            buy = False
            sell_price = buy_num * int(new_dataframe['shortLine'][i])
            account = sell_price * 0.9935 + left
            print("======================")
            print(new_dataframe["날짜"][i])
            print(new_dataframe["시간"][i])
            print("판매가 : ",new_dataframe['shortLine'][i])
            sell_price_all.append(new_dataframe['shortLine'][i])
            buy_price_all.append(new_dataframe['시가'][i])
            print("판매갯수 : ", buy_num)
            print("현재잔고 : ", account)
            print("======================")
        else :
            sell_price_all.append(new_dataframe['시가'][i])
            buy_price_all.append(new_dataframe['시가'][i])


    else:
        sell_price_all.append(new_dataframe['시가'][i])
        buy_price_all.append(new_dataframe['시가'][i])

#         # 사거나 판 날 다음만 비교해야함!!
#         k+1



#     i+=1

#         sell = True

#         short_num = account//int(new_dataframe['shortLine'][i])
#         short_price = int(new_dataframe['shortLine'][i])
#         left = account - short_num * int(new_dataframe['shortLine'][i])
#         print("short",short_num)

sell_price_all.reverse()
buy_price_all.reverse()
print(round(account,2))




20210317
1357
구매가 :  6197.359464500001
구매갯수 :  1613.0
현재가격 : 9995647.122200001
나머지 :  4352.877799998969
20210318
1144
판매가 :  6354.86
판매갯수 :  1613.0
현재잔고 :  10186736.364799999
20210318
1148
구매가 :  6437.395464500001
구매갯수 :  1582.0
현재가격 : 10183279.2628
나머지 :  3457.1019999980927
20210319
903
판매가 :  7834.58
판매갯수 :  1582.0
현재잔고 :  12316288.079999998
20210322
1226
구매가 :  6787.447964500001
구매갯수 :  1814.0
현재가격 : 12311650.470600002
나머지 :  4637.609399996698
20210325
1457
판매가 :  6664.86
판매갯수 :  1814.0
현재잔고 :  12014558.385399997
20210329
1251
구매가 :  6867.459964500001
구매갯수 :  1749.0
현재가격 : 12010435.295100002
나머지 :  4123.090299995616
20210331
1206
판매가 :  6794.86
판매갯수 :  1749.0
현재잔고 :  11809591.501299996
20210331
1431
구매가 :  6799.829821500001
구매갯수 :  1736.0
현재가격 : 11803098.1992
나머지 :  6493.302099995315
20210405
1251
판매가 :  7299.44
판매갯수 :  1736.0
현재잔고 :  12595195.386099996
20210409
1013
구매가 :  9374.265929000001
구매갯수 :  1343.0
현재가격 : 12588483.989400001
나머지 :  6711.3966999948025
20210423
957
판매가 :  16672

NameError: name 'stock' is not defined